## Imports

In [1]:
from game import GAME
import torch
import numpy as np

import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass

from torch.distributions import Normal



from curriculum.reset import reset_random_one_ball

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Instanciando game e exemplificando

In [2]:
env = GAME(draw=True)  
observations = env.reset()

In [3]:
def rewards_function(information):
    colisoes = information['colisoes']
    bolas_caidas = information['bolas_caidas']
    perdeu = information['perdeu']
    ganhou = information['ganhou']
    joga_novamente = information['joga_novamente']
    bolas_jogador = information.get('bolas_jogador', [])
    bolas_adversario = information.get('bolas_adversario', [])
    winner = information.get('winner', None)
    penalizado = information.get('penalizado', False)

    rewards = 0

    if joga_novamente:
        rewards += 1

    if perdeu:
        rewards -= 1.5

    if ganhou:
        rewards += 1.5

    if penalizado:
        rewards -= 1

    return rewards

In [4]:
env.table.draw()

In [5]:
## jogador faz a jogada


env.iniciou_jogada = False
while not env.iniciou_jogada:
    env.table.draw()

obs, information, terminations, rewards = env.step((env.iniciou_jogada_angulo, 
                                                    env.inicou_jogada_intensidade), rewards_function=rewards_function)

print('state', obs)
print('termination', terminations)
print('rewards', rewards)

state (tensor([[0.0137, 0.5000, 1.0000, 0.0000],
        [0.7535, 0.7476, 1.0000, 0.0000],
        [0.7545, 0.4865, 1.0000, 0.0000],
        [0.9280, 0.2381, 1.0000, 0.0000],
        [0.7937, 0.4062, 1.0000, 0.0000],
        [0.8259, 0.5281, 1.0000, 0.0000],
        [0.8699, 0.6038, 1.0000, 0.0000],
        [0.9671, 0.7087, 1.0000, 0.0000],
        [0.5665, 0.3341, 1.0000, 0.0000],
        [0.7445, 0.2964, 1.0000, 0.0000],
        [0.6428, 0.0969, 1.0000, 0.0000],
        [0.8062, 0.4619, 1.0000, 0.0000],
        [0.7291, 0.4671, 1.0000, 0.0000],
        [0.9655, 0.8755, 1.0000, 0.0000],
        [0.8250, 0.3750, 1.0000, 0.0000]]), tensor([0.7510, 0.5762]))
termination False
rewards 0


In [6]:
# computador faz a jogada
print("Iniciando jogada", env.jogador_atual)



angulo = 10
intensidade = 0.9

env.iniciou_jogada = False
obs, information, terminations, rewards = env.step( (angulo, intensidade),  rewards_function=rewards_function)

print("Recompensa da jogada:", rewards)

Iniciando jogada True
Recompensa da jogada: 0


## Começando treinamento
                           

### ARGS

In [7]:
from models.model_transformers import Args, TransformerValueModel, TransformersAtor

### Modelo política

A ideia é que tenhamos uma rede neural como política, especificamente um transformers. 

Queremos que ele passe por cada bola e gere uma representação, no fim a representação do nosso estado será o embedding dabola branca.

#### Instanciando e testando modelo

In [8]:
args = Args( )
model = TransformersAtor( args )

Exemplo de foward:

In [9]:
state, bola_branca = (torch.tensor([
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]
        ]),
                      
 torch.tensor([[0.2000, 0.5000],[0.2000, 0.5000]]))

model(state,bola_branca)

tensor([[303.6469,   0.5058],
        [313.9930,   0.4833]], grad_fn=<CatBackward0>)

In [10]:
state, bola_branca = (torch.tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.7312, 0.4688, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]]), torch.tensor([[0.6916, 0.8642]]))

print(state.shape, bola_branca.shape)
model(state,bola_branca)

torch.Size([1, 15, 4]) torch.Size([1, 2])


tensor([[325.0546,   0.4613]], grad_fn=<CatBackward0>)

### Value model

In [11]:
model = TransformerValueModel( args )

### PPO

In [122]:
class Agent(nn.Module):
    def __init__(self, action_dim, args):
        super().__init__()
        self.critic = TransformerValueModel(args)  # Modelo para estimar o valor
        self.actor_mean = TransformersAtor(args)  # Modelo para estimar a média das ações
        self.actor_log_std = nn.Parameter(torch.zeros(action_dim))  # Desvio padrão fixo

    def get_value(self, state, bola_branca):
        """Obtém o valor estimado pelo modelo crítico."""
        return self.critic(state, bola_branca)

    def get_action_and_value(self, state, bola_branca, action=None):
        """
        Obtém a ação, log_prob, entropia e valor.
        
        Args:
            x: Entrada para o modelo.
            bola_branca: Dados da bola branca.
            action: Ação opcional para avaliar log_prob e entropia.
        
        Returns:
            action: Ação amostrada ou fornecida.
            log_prob: Logaritmo da probabilidade da ação.
            entropy: Entropia da distribuição.
            value: Valor estimado pelo crítico.
        """
        # O ator retorna ângulo e intensidade
        mean = self.actor_mean(state, bola_branca)  # Predição da média das ações
                
        log_std = self.actor_log_std.expand_as(mean)  # Ajusta o desvio padrão ao formato correto
        std = torch.exp(log_std)  # Calcula o desvio padrão

        # Distribuição Normal para ações contínuas
        dist = Normal(mean, std)

        if action is None:
            action = dist.sample()  # Amostra uma ação da distribuição

        action[..., 0] = torch.clamp(action[..., 0], 0, 360)
        action[..., 1] = torch.clamp(action[..., 1], 0, 1)
    
        log_prob = dist.log_prob(action).sum(axis=-1)  # Log probabilidade
        entropy = dist.entropy().sum(axis=-1)  # Entropia

        value = self.critic(state, bola_branca)  # Valor estimado pelo crítico

        return action, log_prob, entropy, value

In [172]:
action_dim = 2

In [173]:
agent = Agent(action_dim, args)

torch.Size([2, 2])

In [180]:
action, log_prob, entropy, value = agent.get_action_and_value(torch.concat((state,state)),torch.concat((bola_branca,bola_branca)))

print('action', action)
print('log_prob', log_prob)
print('entropy', entropy)
print('value', value)

action tensor([[ 32.0625,   0.5783],
        [101.6929,   0.6640]])
log_prob tensor([-3.2769, -1.8512], grad_fn=<SumBackward1>)
entropy tensor([2.8379, 2.8379], grad_fn=<SumBackward1>)
value tensor([[0.0828],
        [0.0119]], grad_fn=<AddmmBackward0>)


Aqui vemos que as ações não estão no range esperado, isso porque estamos passando uma distribuiçao normal 

In [54]:
class PPO:
    def __init__(self, policy_model, value_model ,lr=1e-3, gamma=0.99, eps_clip=0.2, entropy_coef=0.01):
        self.policy = policy_model
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.entropy_coef = entropy_coef
        
        self.value_model = value_model
        self.value_optimizer = optim.Adam(self.value_model.parameters(), lr=lr)
        
    def compute_advantage(self, rewards, values, masks, gamma):
        returns = []
        R = 0
        for reward, mask in zip(reversed(rewards), reversed(masks)):
            R = reward + gamma * R * mask
            returns.insert(0, R)
        returns = torch.tensor(returns, dtype=torch.float32)
        advantage = returns - values
        return advantage.detach(), returns
    
    def update(self, states, bola_branca, actions, log_probs_old, rewards, dones):
        states_flat = states.view(states.size(0), -1)
        batch_size = states.size(0)

        # Calculando valores e vantagens
        values = self.value_model(states_flat, bola_branca).squeeze(-1)
        advantages, returns = self.compute_advantage(rewards, values, 1 - dones, self.gamma)
        
        # Atualizar política
        for _ in range(4):  # Número de atualizações
            outputs = self.policy(states, bola_branca)
            dist = Normal(outputs, torch.ones_like(outputs))  # Normal para distribuição
            log_probs = dist.log_prob(actions).sum(-1)
            entropy = dist.entropy().sum(-1)
            
            # Clipped Surrogate Objective
            ratio = torch.exp(log_probs - log_probs_old)
            surrogate1 = ratio * advantages
            surrogate2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            policy_loss = -torch.min(surrogate1, surrogate2).mean()
            entropy_loss = -self.entropy_coef * entropy.mean()
            
            self.optimizer.zero_grad()
            (policy_loss + entropy_loss).backward()
            self.optimizer.step()

        # Atualizar função de valor
        for _ in range(4):  # Número de atualizações
            values = self.value_model(torch.cat((states_flat, bola_branca), dim=1)).squeeze(-1)
            value_loss = nn.MSELoss()(values, returns)
            
            self.value_optimizer.zero_grad()
            value_loss.backward()
            self.value_optimizer.step()

In [55]:
policy = Transformers(args)
value = TransformerValueModel(args)

In [56]:
ppo = PPO(policy, value)

In [ ]:
#

cfg.table_x_max - cfg.table_x_min



800

In [ ]:
import utils.config as cfg


reset_random_one_ball(env)
env.table.draw()

In [ ]:

env.table.draw()

In [23]:
state_bolas, state_bola_branca = env.get_observations()
# shuffle the balls
state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])]
state_bolas , state_bola_branca

(tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.5000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]),
 tensor([0.7068, 0.3645]))

In [122]:
def rewards_function(information):
    colisoes = information['colisoes']
    bolas_caidas = information['bolas_caidas']
    perdeu = information['perdeu']
    ganhou = information['ganhou']
    joga_novamente = information['joga_novamente']
    bolas_jogador = information.get('bolas_jogador', [])
    bolas_adversario = information.get('bolas_adversario', [])
    winner = information.get('winner', None)
    penalizado = information.get('penalizado', False)

    rewards = 0

    if penalizado:
        rewards -= 1

    return rewards

In [123]:
n_episodes = 1000  # Número de episódios para o treinamento
max_steps = 200  # Passos máximos por episódio
gamma = 0.99  # Fator de desconto
learning_rate = 1e-3

In [124]:
agent = Agent(action_dim=2, args=args)  # Dois valores de ação (ângulo, força)
ppo = PPO(policy_model=agent, value_model=agent.critic, lr=learning_rate, gamma=gamma)

In [125]:
replay_buffer = {
    "states": [],
    "actions": [],
    "log_probs": [],
    "rewards": [],
    "dones": [],
    "bola_branca": []
}

(tensor([[0.8250, 0.5625, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]),
 tensor([0.0312, 0.7628]))

In [126]:
agent.get_action_and_value(state, bola_branca)

(tensor([[135.2115,   2.5012]]),
 tensor([-5.0424], grad_fn=<SumBackward1>),
 tensor([2.8379], grad_fn=<SumBackward1>),
 tensor([[-0.3033]], grad_fn=<AddmmBackward0>))

In [127]:
state, bola_branca

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.7312, 0.4688, 1.0000, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]]]),
 tensor([[0.6916, 0.8642]]))

In [128]:
state_bolas, state_bola_branca

(tensor([[0.7625, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.5938, 1.0000, 0.0000],
         [0.8250, 0.6250, 1.0000, 0.0000],
         [0.0137, 0.3837, 1.0000, 0.0000],
         [0.8250, 0.4375, 1.0000, 0.0000],
         [0.7625, 0.4375, 1.0000, 0.0000],
         [0.8250, 0.5000, 1.0000, 0.0000],
         [0.7937, 0.5312, 1.0000, 0.0000],
         [0.7625, 0.5000, 1.0000, 0.0000],
         [0.7312, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 0.0000],
         [0.7312, 0.5312, 1.0000, 0.0000],
         [0.7937, 0.4062, 1.0000, 0.0000]]),
 tensor([0.8709, 0.3837]))

In [129]:
print(state_bolas.shape, state_bola_branca.shape)

torch.Size([15, 4]) torch.Size([2])


In [ ]:
def train():
    
    for episode in range(n_episodes):
        
        reset_random_one_ball(env=env, raio_buraco=13)
        
        rewards_sum = 0
        for step in range(max_steps):
            # Amostra uma ação do agente
            
            
            state_bolas , state_bola_branca = env.get_observations()
            
            action, log_prob, entropy, value = agent.get_action_and_value(state_bolas, state_bola_branca)
            # Executa a ação no ambiente
            
            print(action)
            
            angulo = action[0, 0].item()
            intensidade = action[0, 1].item()
            
            next_obs, information, done, reward = env.step((angulo, intensidade), rewards_function=rewards_function)

            next_state_bolas, next_state_bola_branca = next_obs
            
            # Armazenar dados no buffer
            replay_buffer["states"].append(state_bolas)
            replay_buffer["actions"].append(action)
            replay_buffer["log_probs"].append(log_prob)
            replay_buffer["rewards"].append(reward)
            replay_buffer["dones"].append(done)
            replay_buffer["bola_branca"].append(state_bola_branca)

            print(rewards_sum, reward)
            rewards_sum += reward

            # Atualizar estados
            state_bolas = next_state_bolas
            state_bola_branca = next_state_bola_branca

            if done:
                break
        
        # Atualizar a política usando os dados do episódio
        ppo.update(
            states=torch.stack(replay_buffer["states"]),
            bola_branca=torch.stack(replay_buffer["bola_branca"]),
            actions=torch.stack(replay_buffer["actions"]),
            log_probs_old=torch.stack(replay_buffer["log_probs"]),
            rewards=torch.tensor(replay_buffer["rewards"]),
            dones=torch.tensor(replay_buffer["dones"])
        )
        
        # Limpar o buffer para o próximo episódio
        for key in replay_buffer.keys():
            replay_buffer[key] = []

        print(f"Episódio {episode + 1}/{n_episodes}, Recompensa Total: {rewards_sum}")

In [131]:
train()

/Users/artur/Documents/VsCode/8pool-with-reinforcement-learning/utils/Ball.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.posicao = torch.tensor(posicao, device=cfg.device, dtype=torch.float32)


tensor([[2.1050e+02, 1.0058e-01]])
0 -1


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [87]:
env.reset()
state_bolas, state_bola_branca = env.get_observations()
env.table.bolas = env.table.bolas[1:2] + [env.table.bola_branca]
numero_bola = env.table.bolas[0].numero
state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])] 

state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])]        
        
env.table.bolas = env.table.bolas[1:2] + [env.table.bola_branca]

In [88]:
state_bolas, state_bola_branca

(tensor([[0.7625, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.5625, 1.0000, 0.0000],
         [0.7937, 0.5938, 1.0000, 0.0000],
         [0.8250, 0.6250, 1.0000, 0.0000],
         [0.0137, 0.3837, 1.0000, 0.0000],
         [0.8250, 0.4375, 1.0000, 0.0000],
         [0.7625, 0.4375, 1.0000, 0.0000],
         [0.8250, 0.5000, 1.0000, 0.0000],
         [0.7937, 0.5312, 1.0000, 0.0000],
         [0.7625, 0.5000, 1.0000, 0.0000],
         [0.7312, 0.4688, 1.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 0.0000],
         [0.7312, 0.5312, 1.0000, 0.0000],
         [0.7937, 0.4062, 1.0000, 0.0000]]),
 tensor([0.8709, 0.3837]))

In [ ]:
torch.tensor([[352.0620,   1.3407]])

RuntimeError: values expected sparse tensor layout but got Strided

In [132]:
print("Recompensas no buffer:", replay_buffer["rewards"])

Recompensas no buffer: [-1]
